In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
rating_data = pd.read_csv("data/animelist.csv")

In [9]:
rating_data

,Unnamed: 0,user_id,anime_id,rating,watching_status,watched_episodes
0,213,2,24833,0,1,0
1,214,2,235,10,1,100
2,215,2,36721,0,1,1
3,216,2,40956,0,1,0
4,217,2,31933,0,1,3
...,...,...,...,...,...,...
6529912,9999995,32600,6,0,3,2
6529913,9999996,32600,2129,4,3,2
6529914,9999997,32600,34902,0,3,4
6529915,9999998,32600,37722,0,3,1


In [22]:
min_rating = min(rating_data['rating'])
max_rating = max(rating_data['rating'])
rating_data['rating'] = rating_data["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values.astype(np.float64)

AvgRating = np.mean(rating_data['rating'])
print('Avg', AvgRating)

Avg 0.4073779191986666


In [23]:
duplicates = rating_data.duplicated()

if duplicates.sum() > 0:
    print('> {} duplicates'.format(duplicates.sum()))
    rating_data = rating_data[~duplicates]

print('> {} duplicates'.format(rating_data.duplicated().sum()))

> 0 duplicates


In [24]:
rating_data.isna().sum()

Unnamed: 0          0
user_id             0
anime_id            0
rating              0
watching_status     0
watched_episodes    0
user                0
anime               0
dtype: int64

In [25]:
g = rating_data.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rating_data.join(top_users, rsuffix='_r', how='inner', on='user_id')
top_r

,Unnamed: 0,user_id,anime_id,rating,watching_status,watched_episodes,user,anime,rating_r
865144,1374764,4773,40679,0.0,1,9,1166,4949,5735
865145,1374765,4773,39783,0.0,1,9,1166,3534,5735
865146,1374766,4773,44040,0.0,1,9,1166,9395,5735
865147,1374767,4773,42514,0.0,1,21,1166,11791,5735
865148,1374768,4773,40930,0.0,1,8,1166,5347,5735
...,...,...,...,...,...,...,...,...,...
6155402,9429599,30761,37976,0.0,6,0,7883,2155,4571
6155403,9429600,30761,40174,0.0,6,0,7883,5015,4571
6155404,9429601,30761,1248,0.0,6,0,7883,6233,4571
6155405,9429602,30761,1249,0.0,6,0,7883,6277,4571


In [28]:
# Encoding categorical data
user_ids = rating_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_data["user"] = rating_data["user_id"].map(user2user_encoded)
n_users = len(user2user_encoded)

anime_ids = rating_data["anime_id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_data["anime"] = rating_data["anime_id"].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min rating: {}, Max rating: {}".format(min(rating_data['rating']), max(rating_data['rating'])))

Num of users: 8348, Num of animes: 17554
Min rating: 0.0, Max rating: 1.0


In [35]:
rating_data = rating_data.drop(columns="Unnamed: 0")
rating_data

,user_id,anime_id,rating,watching_status,watched_episodes,user,anime
0,2,24833,0.0,1,0,0,0
1,2,235,1.0,1,100,0,1
2,2,36721,0.0,1,1,0,2
3,2,40956,0.0,1,0,0,3
4,2,31933,0.0,1,3,0,4
...,...,...,...,...,...,...,...
6529912,32600,6,0.0,3,2,8347,1000
6529913,32600,2129,0.4,3,2,8347,2442
6529914,32600,34902,0.0,3,4,8347,1587
6529915,32600,37722,0.0,3,1,8347,2767
